In [1]:
from benzinga import news_data
import config 
news = news_data.News(config.api_key)

In [10]:
stories = news.news(date_from = "2015-01-01", date_to= "2023-01-01", pagesize=100000)

2024-02-19 16:21:16 [info     ] Status Code: 200 Endpoint: http://api.benzinga.com/api/v2/news/?token=6a2133b56c284accbb1d1f7ae8be9152&pageSize=100000&dateFrom=2015-01-01&dateTo=2023-01-01


In [11]:
stories

[{'id': 5113793,
  'author': 'Benzinga',
  'created': 'Thu, 01 Jan 2015 15:38:08 -0400',
  'updated': 'Thu, 01 Jan 2015 15:38:08 -0400',
  'title': "Analysts Generally Bullish On New Year's Eve: Here Are Some Trading Ideas For 2015",
  'teaser': '',
  'body': '',
  'url': 'https://www.benzinga.com/node/5113793',
  'image': [],
  'channels': [{'name': 'Long Ideas'},
   {'name': 'News'},
   {'name': 'Upgrades'},
   {'name': 'Price Target'},
   {'name': 'Reiteration'},
   {'name': 'Hot'},
   {'name': 'Analyst Ratings'},
   {'name': 'Trading Ideas'}],
  'stocks': [{'name': 'CEMP'},
   {'name': 'ESL'},
   {'name': 'HON'},
   {'name': 'MEOH'},
   {'name': 'SWN'}],
  'tags': [{'name': 'Capital One Financial'},
   {'name': 'imperial capital'},
   {'name': 'Jefferies'},
   {'name': 'JP Morgan'},
   {'name': 'Monness Crespi Hardt'},
   {'name': 'Stifel Nicolaus'},
   {'name': 'TD Securities'},
   {'name': 'UBS'}]},
 {'id': 5113793,
  'author': 'Benzinga',
  'created': 'Thu, 01 Jan 2015 15:38:08 

In [12]:
headlines = [item['title'] for item in stories]
len(headlines)

100

In [4]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
  
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

/Users/ravipanguluri/miniconda3/envs/portfolio-rl/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
pytorch_model.bin: 100%|██████████| 438M/438M [00:26<00:00, 16.3MB/s] 


In [13]:
inputs = tokenizer(headlines, padding = True, truncation = True, return_tensors='pt')
print(inputs)

{'input_ids': tensor([[  101, 18288,  3227,  ...,     0,     0,     0],
        [  101, 18288,  3227,  ...,     0,     0,     0],
        [  101, 25294, 24273,  ...,     0,     0,     0],
        ...,
        [  101,  5076, 28077,  ...,     0,     0,     0],
        [  101,  4422,  2061,  ...,     0,     0,     0],
        [  101,  4422,  2061,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}


In [14]:
outputs = model(**inputs)
print(outputs.logits.shape)

torch.Size([100, 3])


In [15]:
import torch
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
print(predictions)

tensor([[0.0358, 0.0436, 0.9206],
        [0.0358, 0.0436, 0.9206],
        [0.9521, 0.0178, 0.0301],
        [0.9521, 0.0178, 0.0301],
        [0.3189, 0.0136, 0.6675],
        [0.3189, 0.0136, 0.6675],
        [0.3189, 0.0136, 0.6675],
        [0.0569, 0.3792, 0.5639],
        [0.0569, 0.3792, 0.5639],
        [0.0569, 0.3792, 0.5639],
        [0.0899, 0.1423, 0.7679],
        [0.0899, 0.1423, 0.7679],
        [0.0667, 0.0117, 0.9216],
        [0.0667, 0.0117, 0.9216],
        [0.2920, 0.0111, 0.6969],
        [0.2920, 0.0111, 0.6969],
        [0.2920, 0.0111, 0.6969],
        [0.0297, 0.0259, 0.9444],
        [0.0297, 0.0259, 0.9444],
        [0.0459, 0.4312, 0.5229],
        [0.0459, 0.4312, 0.5229],
        [0.0503, 0.8993, 0.0504],
        [0.0503, 0.8993, 0.0504],
        [0.0503, 0.8993, 0.0504],
        [0.9488, 0.0245, 0.0266],
        [0.9488, 0.0245, 0.0266],
        [0.8874, 0.0087, 0.1039],
        [0.8874, 0.0087, 0.1039],
        [0.8874, 0.0087, 0.1039],
        [0.043

In [16]:
import pandas as pd


positive = predictions[:, 0].tolist()
negative = predictions[:, 1].tolist()
neutral = predictions[:, 2].tolist()


table = {'Headline':headlines,
         "Positive":positive,
         "Negative":negative, 
         "Neutral":neutral}
      
df = pd.DataFrame(table, columns = ["Headline", "Positive", "Negative", "Neutral"])


In [17]:
df

,Headline,Positive,Negative,Neutral
0,Analysts Generally Bullish On New Year's Eve: ...,0.035752,0.043607,0.920641
1,Analysts Generally Bullish On New Year's Eve: ...,0.035752,0.043607,0.920641
2,Argus Expects Apple To Report Strong Q1 Result...,0.952074,0.017783,0.030143
3,Argus Expects Apple To Report Strong Q1 Result...,0.952074,0.017783,0.030143
4,Bitcoin Poised For Success In 2015?,0.318920,0.013598,0.667482
...,...,...,...,...
95,Benzinga's Volume Movers,0.055503,0.040972,0.903525
96,"Jefferies: Wal-Mart Saw Good Holiday Sales, Lo...",0.949294,0.027277,0.023429
97,"Jefferies: Wal-Mart Saw Good Holiday Sales, Lo...",0.949294,0.027277,0.023429
98,Brian Sozzi On JC Penney's Q4 Report,0.030968,0.037986,0.931046
